In [17]:
import numpy as np

In [18]:
# Import phi from train data set from 1 st row to 14 rows. (Excluding 14 th column)
phi = np.loadtxt('data/train.csv', dtype='float', delimiter=',', skiprows=1, usecols=tuple(range(1, 14)))

In [19]:
# Import y from train data set by using last column
y = np.loadtxt('data/train.csv', dtype='float', delimiter=',', skiprows=1, usecols=14, ndmin=2)

In [20]:
# Import phi_test from test data set
phi_test = np.loadtxt('data/test.csv', dtype='float', delimiter=',', skiprows=1, usecols=tuple(range(1, 14)))


In [21]:
# Add a cloloumn of 1s to right of phi and phi_test
phi_test = np.concatenate((phi_test, np.ones((105, 1))), axis=1)
phi = np.concatenate((phi, np.ones((400, 1))), axis=1)

In [22]:
# Apply min max scaling on each coloumn of phi and phi_test

for i in range(0, 13):
    col_max = max(phi[:, i])
    col_min = min(phi[:, i])
    phi[:, i] = (phi[:, i] - col_min) / (col_max - col_min)
    phi_test[:, i] = (phi_test[:, i] - col_min) / (col_max - col_min)

In [23]:
# Log scaling on y
y = np.log(y)

In [24]:
# Define a function to calculate change in error function based on phi, w and p norm
def change_error(p, phi, w):
    if p == 2:
        change_err = (2 * (np.dot(np.dot(np.transpose(phi), phi), w) - np.dot(np.transpose(phi), y)) + lambd * p * np.power(np.absolute(w), (p - 1)))
    if p < 2 and p > 1:
        change_err = (2 * (np.dot(np.dot(np.transpose(phi), phi), w) - np.dot(np.transpose(phi), y)) + lambd * p * np.power(np.absolute(w), (p - 1)) * np.sign(w))
    return change_err

In [25]:
# Make a dictionary containing filenames as keys and p as values
filenames = {'out.csv': 2.0,
             'out_1.csv': 1.75,
             'out_2.csv': 1.5,
             'out_3.csv': 1.3
             }

In [27]:
# For each item in this dictionary
for (fname, p) in filenames.items():
    # Set initial w to all zeros
    w = np.zeros((14, 1))
    # Hyperparameter lambda value
    lambd = 0.2

    # Maximum step size
    t = 0.00012

    # Calculate new value of w
    w_new = w - t * change_error(p, phi, w)

    i = 0
    # Repeat steps until error between consecutive w is less than threshold
    while(np.linalg.norm(w_new-w) > 10 ** -10):
        w = w_new
        w_new = w - t * change_error(p, phi, w)
        i = i + 1

    # Load values of id
    id_test = np.loadtxt('data/test.csv', dtype='int', delimiter=',', skiprows=1, usecols=0, ndmin=2)

    # Calculate y for test data
    y_test = np.exp(np.dot(phi_test, w_new))

    # Save the ids and y
    np.savetxt(fname, np.concatenate((id_test, y_test), axis=1), delimiter=',', fmt=['%d', '%f'], header='ID,MEDV', comments='')
